In [1]:
# stp 1 Dependecis instal
#!pip install transformers==4.41.2
# Important to install that transformer version, unles it occur error
#See link: https://github.com/THUDM/CogVLM2/issues/181#issuecomment-2381807778
# !pip install transformers sentencepiece datasets evaluate
# !pip install llmlingua

In [2]:
import torch
import json
import time
import os
from transformers import AutoModel, AutoTokenizer, GenerationConfig
from datasets import load_dataset
from tqdm.notebook import tqdm 
import numpy as np
from sklearn.metrics import f1_score
#from LLMLingua.experiments.llmlingua2.evaluation.metrics import qa_f1_score
from metrics import qa_f1_score, f1_score
if 'model' in globals():
    del model 
torch.cuda.empty_cache()

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

#######################################
print('cuda is',torch.cuda.is_available()) 
#################################################
# 3 model & dataset 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cuda is True


In [3]:
print('cuda is',torch.cuda.is_available()) 

cuda is True


In [4]:
# ChatGLM-6B
model = AutoModel.from_pretrained(
    "THUDM/chatglm2-6b-32k",
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto",
    use_cache=True
).to(device)

model=model.eval()
#print('model config is:',model.generation_config)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b-32k", trust_remote_code=True)#.to(device)

In [6]:
# Monkey-patch the tokenizer's _pad method to handle the padding_side argument
original_pad = tokenizer._pad
def new_pad(self, encoded_inputs, max_length=None, padding_strategy="longest", pad_to_multiple_of=None, return_attention_mask=None, **kwargs):
    return original_pad(encoded_inputs, max_length, padding_strategy, pad_to_multiple_of, return_attention_mask)
tokenizer._pad = new_pad.__get__(tokenizer, type(tokenizer))


In [7]:
# Load the four datasets
datasets = ["multifieldqa_en", "hotpotqa", "triviaqa", "narrativeqa"]

In [8]:
#RANDOM token PRUNE
def random_prune(text, compression_ratio=0.5):

    tokens = tokenizer.tokenize(text)#.to(device)  # e.g., ["The", " quick", " brown"...]
    
    num_keep = int(len(tokens) * (compression_ratio)) # compression ratio means remaining 
    
    # Step 3: Randomly pick tokens (like lottery balls)
    kept_indices = sorted(  # Maintain original order
        np.random.choice(  # Random selection
            len(tokens), 
            num_keep, 
            replace=False  # No duplicates
        )
    )
    
    # Step 4: Rebuild text from kept tokens
    pruned_tokens = [tokens[i] for i in kept_indices]
    return tokenizer.convert_tokens_to_string(pruned_tokens)  # Tokens ➔ text

In [9]:
#Baseline

def no_pruning(context: str) -> str:
    """Returns the full context (no pruning)."""
    return context


In [10]:
def evaluate_multifield(data, compression_func, ratio=None):
    f1_scores = []
    latencies = []
    memory_usages = []
    peak_memory_usages = []

    #for idx in range(20): 
    for idx in range(len(data)):  # Test on all examples
        example = data[idx] 
        context = example["context"]
        question = example["input"]
        answers =example["answers"][0]
        if ratio is not None:
            compressed_context = compression_func(context, ratio)
        else:
            compressed_context = compression_func(context)  #for no.prune & compressr
            
        #prompt = f"Context: {compressed_context}\nQuestion: {question}"
        prompt = f"Read the following text and answer briefly.\n\n{compressed_context}\n\nNow, answer the following question based on the above text, only give me the answer and do not output any other words.\n\nQuestion: {question}\nAnswer:"
        torch.cuda.reset_peak_memory_stats()
        start_time = time.time()
        response, _ = model.chat(tokenizer, prompt)
        # response, _ = model.chat(
        #     tokenizer,
        #     question,
        #     history=[(truncated_context, "Answer:")],
        #     max_new_tokens=max_new_tokens  # Critical parameter
        # )
        latency = time.time() - start_time
        
        # Measure memory
        memory = torch.cuda.memory_allocated() / (1024 ** 2)  # MB
        
        #Measure peak memory?
        peak_memory = torch.cuda.max_memory_allocated() / (1024 ** 2)
        
        # Calculate F1 score
        f1 = qa_f1_score(prediction=response, ground_truth=answers)
        f1_scores.append(f1)
        latencies.append(latency)
        peak_memory_usages.append(peak_memory)
    
    return {
        "avg_f1": np.mean(f1_scores),
        "avg_latency": np.mean(latencies),
        "avg_peak_memory": np.mean(peak_memory_usages)
    }

In [11]:
def evaluate_hotpot(data, compression_func, ratio=None):
    f1_scores = []
    latencies = []
    memory_usages = []
    peak_memory_usages = []

    #for idx in range(20): 
    for idx in range(len(data)):  # Test on all examples
        example = data[idx] 
        context = example["context"]
        question = example["input"]
        answers =example["answers"][0]
        if ratio is not None:
            compressed_context = compression_func(context, ratio)
        else:
            compressed_context = compression_func(context)  #for no.prune & compressr
            
        prompt = f"Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{compressed_context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {question}\nAnswer:"
        torch.cuda.reset_peak_memory_stats()
        start_time = time.time()
        response, _ = model.chat(tokenizer, prompt)
        latency = time.time() - start_time
        
        # Measure memory
        memory = torch.cuda.memory_allocated() / (1024 ** 2)  # MB
        
        #Measure peak memory?
        peak_memory = torch.cuda.max_memory_allocated() / (1024 ** 2)
        
        # Calculate F1 score
        f1 = qa_f1_score(prediction=response, ground_truth=answers)
        f1_scores.append(f1)
        latencies.append(latency)
        peak_memory_usages.append(peak_memory)
    
    return {
        "avg_f1": np.mean(f1_scores),
        "avg_latency": np.mean(latencies),
        "avg_peak_memory": np.mean(peak_memory_usages)
    }

In [12]:
def evaluate_trivia(data, compression_func, ratio=None):
    f1_scores = []
    latencies = []
    memory_usages = []
    peak_memory_usages = []

    #for idx in range(20): 
    for idx in range(len(data)):  # Test on all examples
        example = data[idx] 
        context = example["context"]
        question = example["input"]
        answers =example["answers"][0]
        if ratio is not None:
            compressed_context = compression_func(context, ratio)
        else:
            compressed_context = compression_func(context)  #for no.prune & compressr
            
        prompt = f"Answer the question based on the given passage. Only give me the answer and do not output any other words. The following are some examples.\n\n{compressed_context}\n\n{question}"
        torch.cuda.reset_peak_memory_stats()
        start_time = time.time()
        response, _ = model.chat(tokenizer, prompt)
        latency = time.time() - start_time
        
        # Measure memory
        memory = torch.cuda.memory_allocated() / (1024 ** 2)  # MB
        
        #Measure peak memory?
        peak_memory = torch.cuda.max_memory_allocated() / (1024 ** 2)
        
        # Calculate F1 score
        f1 = qa_f1_score(prediction=response, ground_truth=answers)
        f1_scores.append(f1)
        latencies.append(latency)
        peak_memory_usages.append(peak_memory)
    
    return {
        "avg_f1": np.mean(f1_scores),
        "avg_latency": np.mean(latencies),
        "avg_peak_memory": np.mean(peak_memory_usages)
    }

In [24]:
def evaluate_narrative(data, compression_func, ratio=None):
    f1_scores = []
    latencies = []
    peak_memory_usages = []
    
    # Model constraints (ChatGLM2-6B-32k actual limits)
    max_context_length = 32768  # Actual model limit
    max_new_tokens = 256        # Space for response generation
    max_input_length = max_context_length - max_new_tokens  # 32512

    for idx in range(len(data)):
        example = data[idx] 
        context = example["context"]
        question = example["input"]
        answers = example["answers"][0]
        
        # Convert answer to string if needed
        if isinstance(answers, list):
            answers = " ".join(answers)
        answers = str(answers)

        try:
            # Apply compression
            if ratio is not None:
                compressed_context = compression_func(context, ratio)
            else:
                compressed_context = compression_func(context)

            # Tokenize and truncate to model limits
            tokenized = tokenizer.encode(
                compressed_context,
                truncation=True,
                max_length=max_input_length,
                return_tensors="pt"
            )
            truncated_context = tokenizer.decode(tokenized[0], skip_special_tokens=True)

            # Build final prompt
            prompt = f"You are given a story...\n\nQuestion: {question}\n\nAnswer:"
            
            # Memory management
            torch.cuda.empty_cache()
            torch.cuda.reset_peak_memory_stats()
            start_time = time.time()
            
            # Generate response with length control
            response, _ = model.chat(
                tokenizer,
                prompt,
                max_new_tokens=max_new_tokens,
                history=[(truncated_context, "Story:")]
            )
            
            latency = time.time() - start_time
            peak_memory = torch.cuda.max_memory_allocated() / (1024 ** 2)

            # Calculate metrics
            response = str(response)  # Ensure string type
            f1 = qa_f1_score(prediction=response, ground_truth=answers)
            
            f1_scores.append(f1)
            latencies.append(latency)
            peak_memory_usages.append(peak_memory)

        except Exception as e:
            print(f"Error processing example {idx}: {str(e)}")
            continue
    
    return {
        "avg_f1": np.mean(f1_scores) if f1_scores else 0,
        "avg_latency": np.mean(latencies) if latencies else 0,
        "avg_peak_memory": np.mean(peak_memory_usages) if peak_memory_usages else 0
    }

In [ ]:
# def evaluate_narrative(data, compression_func, ratio=None):
#     f1_scores = []
#     latencies = []
#     memory_usages = []
#     peak_memory_usages = []
    
#     max_context_length = 40000  # ChatGLM2's limit
#     max_new_tokens = 256        # Reserve space for generation
#     #for idx in range(20): 
#     for idx in range(len(data)):  # Test on all examples
#         example = data[idx] 
#         context = example["context"]
#         question = example["input"]
#         answers =example["answers"][0]
        
#         if ratio is not None:
#             compressed_context = compression_func(context, ratio)
#         else:
#             compressed_context = compression_func(context)  #for no.prune & compressr
            
#         prompt = f"You are given a story, which can be either a novel or a movie script, and a question. Answer the question asconcisely as you can, using a single phrase if possible. Do not provide any explanation.\n\nStory: {compressed_context}\n\nNow, answer the question based on the story asconcisely as you can, using a single phrase if possible. Do not provide any explanation.\n\nQuestion: {question}\n\nAnswer:"
#         torch.cuda.reset_peak_memory_stats()
#         start_time = time.time()
#         response, _ = model.chat(tokenizer, prompt)
#         latency = time.time() - start_time

#         # Measure memory
#         memory = torch.cuda.memory_allocated() / (1024 ** 2)  # MB
        
#         #Measure peak memory?
#         peak_memory = torch.cuda.max_memory_allocated() / (1024 ** 2)
        
#         # Calculate F1 score
#         f1 = qa_f1_score(prediction=response, ground_truth=answers)
#         f1_scores.append(f1)
#         latencies.append(latency)
#         peak_memory_usages.append(peak_memory)
    
#     return {
#         "avg_f1": np.mean(f1_scores),
#         "avg_latency": np.mean(latencies),
#         "avg_peak_memory": np.mean(peak_memory_usages)
#     }

In [14]:
# Reusable function to print and save results
def log_result_no_prune(message):
    #print(message)
    with open("results_no_prune.txt", "a") as f:  # 'a' = append mode
        f.write(message + "\n")

def log_result_random_prune(message):
    #print(message)
    with open("results_random_prune.txt", "a") as f:  # 'a' = append mode
        f.write(message + "\n")

In [15]:
pruning_ratios = [0.25, 0.5, 0.75]
# for dataset_name in datasets:
dataset_name = datasets[0]  # MultifieldQA
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
# Randn pruning
for ratio in pruning_ratios:
    random_pruning_results = evaluate_multifield(data, random_prune, ratio=ratio)
    
    print(f"\n==={dataset_name}  Random Token Pruning ({int(ratio*100)}%) ===")
    print(f"F1: {random_pruning_results['avg_f1']:.2f}, Latency: {random_pruning_results['avg_latency']:.2f}s, Peak Memory: {random_pruning_results['avg_peak_memory']:.2f} MB")
    log_result_random_prune(f"\n==={dataset_name} Random Token Pruning ({int(ratio*100)}%) ===")
    log_result_random_prune(f"F1: {random_pruning_results['avg_f1']:.2f}, Latency: {random_pruning_results['avg_latency']:.2f}s, Peak Memory: {random_pruning_results['avg_peak_memory']:.2f} MB")


===multifieldqa_en  Random Token Pruning (25%) ===
F1: 0.13, Latency: 0.78s, Peak Memory: 12282.26 MB

===multifieldqa_en  Random Token Pruning (50%) ===
F1: 0.20, Latency: 1.13s, Peak Memory: 12607.54 MB

===multifieldqa_en  Random Token Pruning (75%) ===
F1: 0.30, Latency: 1.49s, Peak Memory: 12937.04 MB


In [16]:
pruning_ratios = [0.25, 0.5, 0.75]
# for dataset_name in datasets:
dataset_name = datasets[1]  # HotpotQA
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
# Randn pruning
for ratio in pruning_ratios:
    random_pruning_results = evaluate_hotpot(data, random_prune, ratio=ratio)
    
    print(f"\n==={dataset_name}  Random Token Pruning ({int(ratio*100)}%) ===")
    print(f"F1: {random_pruning_results['avg_f1']:.2f}, Latency: {random_pruning_results['avg_latency']:.2f}s, Peak Memory: {random_pruning_results['avg_peak_memory']:.2f} MB")
    log_result_random_prune(f"\n==={dataset_name} Random Token Pruning ({int(ratio*100)}%) ===")
    log_result_random_prune(f"F1: {random_pruning_results['avg_f1']:.2f}, Latency: {random_pruning_results['avg_latency']:.2f}s, Peak Memory: {random_pruning_results['avg_peak_memory']:.2f} MB")


===hotpotqa  Random Token Pruning (25%) ===
F1: 0.21, Latency: 0.66s, Peak Memory: 12573.86 MB

===hotpotqa  Random Token Pruning (50%) ===
F1: 0.29, Latency: 1.26s, Peak Memory: 13187.23 MB

===hotpotqa  Random Token Pruning (75%) ===
F1: 0.39, Latency: 1.93s, Peak Memory: 13805.52 MB


In [17]:
pruning_ratios = [0.25, 0.5, 0.75]
# for dataset_name in datasets:
dataset_name = datasets[2]  # TriviaQA
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
# Randn pruning
for ratio in pruning_ratios:
    random_pruning_results = evaluate_trivia(data, random_prune, ratio=ratio-0.1)
    
    print(f"\n==={dataset_name}  Random Token Pruning ({int(ratio*100)}%) ===")
    print(f"F1: {random_pruning_results['avg_f1']:.2f}, Latency: {random_pruning_results['avg_latency']:.2f}s, Peak Memory: {random_pruning_results['avg_peak_memory']:.2f} MB")
    log_result_random_prune(f"\n==={dataset_name} Random Token Pruning ({int(ratio*100)}%) ===")
    log_result_random_prune(f"F1: {random_pruning_results['avg_f1']:.2f}, Latency: {random_pruning_results['avg_latency']:.2f}s, Peak Memory: {random_pruning_results['avg_peak_memory']:.2f} MB")


===triviaqa  Random Token Pruning (25%) ===
F1: 0.40, Latency: 0.50s, Peak Memory: 12413.06 MB

===triviaqa  Random Token Pruning (50%) ===
F1: 0.41, Latency: 1.87s, Peak Memory: 12948.16 MB

===triviaqa  Random Token Pruning (75%) ===
F1: 0.42, Latency: 2.93s, Peak Memory: 13487.02 MB


In [25]:
pruning_ratios = [0.25, 0.5, 0.75]
# for dataset_name in datasets:
dataset_name = datasets[3]  # NarrativeQA
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
# Randn pruning
for ratio in pruning_ratios:
    random_pruning_results = evaluate_narrative(data, random_prune, ratio=ratio-0.1)
    
    print(f"\n==={dataset_name}  Random Token Pruning ({int(ratio*100)}%) ===")
    print(f"F1: {random_pruning_results['avg_f1']:.2f}, Latency: {random_pruning_results['avg_latency']:.2f}s, Peak Memory: {random_pruning_results['avg_peak_memory']:.2f} MB")
    log_result_random_prune(f"\n==={dataset_name} Random Token Pruning ({int(ratio*100)}%) ===")
    log_result_random_prune(f"F1: {random_pruning_results['avg_f1']:.2f}, Latency: {random_pruning_results['avg_latency']:.2f}s, Peak Memory: {random_pruning_results['avg_peak_memory']:.2f} MB")

Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_gene


===narrativeqa  Random Token Pruning (25%) ===
F1: 0.09, Latency: 2.24s, Peak Memory: 12844.84 MB


Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_gene


===narrativeqa  Random Token Pruning (50%) ===
F1: 0.12, Latency: 4.38s, Peak Memory: 14323.06 MB


Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_gene


===narrativeqa  Random Token Pruning (75%) ===
F1: 0.16, Latency: 9.04s, Peak Memory: 15366.91 MB


In [26]:
# for dataset_name in datasets:
dataset_name = datasets[0]  # MultifieldQA
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
# Evaluate no pruning (full context)
full_context_results = evaluate_multifield(data, no_pruning)
print(f"\n==={dataset_name}  No Pruning (Full Context) ===")
print(f"F1: {full_context_results['avg_f1']:.2f}, Latency: {full_context_results['avg_latency']:.2f}s,  Peak Memory: {full_context_results['avg_peak_memory']:.2f} MB")
log_result_no_prune(f"\n==={dataset_name} No Pruning (Full Context) ===")
log_result_no_prune(f"F1: {full_context_results['avg_f1']:.2f}, Latency: {full_context_results['avg_latency']:.2f}s, Peak Memory: {full_context_results['avg_peak_memory']:.2f} MB")


===multifieldqa_en  No Pruning (Full Context) ===
F1: 0.44, Latency: 3.75s,  Peak Memory: 13274.23 MB


In [27]:
# for dataset_name in datasets:
dataset_name = datasets[1]  # HotpotQA
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
# Evaluate no pruning (full context)
full_context_results = evaluate_hotpot(data, no_pruning)
print(f"\n==={dataset_name}  No Pruning (Full Context) ===")
print(f"F1: {full_context_results['avg_f1']:.2f}, Latency: {full_context_results['avg_latency']:.2f}s,  Peak Memory: {full_context_results['avg_peak_memory']:.2f} MB")
log_result_no_prune(f"\n==={dataset_name} No Pruning (Full Context) ===")
log_result_no_prune(f"F1: {full_context_results['avg_f1']:.2f}, Latency: {full_context_results['avg_latency']:.2f}s, Peak Memory: {full_context_results['avg_peak_memory']:.2f} MB")


===hotpotqa  No Pruning (Full Context) ===
F1: 0.48, Latency: 5.45s,  Peak Memory: 14430.83 MB


In [28]:
# for dataset_name in datasets:
dataset_name = datasets[2]  # TriviaQA
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
# Evaluate no pruning (full context)
full_context_results = evaluate_trivia(data, no_pruning)
print(f"\n==={dataset_name}  No Pruning (Full Context) ===")
print(f"F1: {full_context_results['avg_f1']:.2f}, Latency: {full_context_results['avg_latency']:.2f}s,  Peak Memory: {full_context_results['avg_peak_memory']:.2f} MB")
log_result_no_prune(f"\n==={dataset_name} No Pruning (Full Context) ===")
log_result_no_prune(f"F1: {full_context_results['avg_f1']:.2f}, Latency: {full_context_results['avg_latency']:.2f}s, Peak Memory: {full_context_results['avg_peak_memory']:.2f} MB")


===triviaqa  No Pruning (Full Context) ===
F1: 0.41, Latency: 5.37s,  Peak Memory: 14251.10 MB


In [29]:
# for dataset_name in datasets:
dataset_name = datasets[3]  # NarrativeQA
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
# Evaluate no pruning (full context)
full_context_results = evaluate_narrative(data, no_pruning)
print(f"\n==={dataset_name}  No Pruning (Full Context) ===")
print(f"F1: {full_context_results['avg_f1']:.2f}, Latency: {full_context_results['avg_latency']:.2f}s,  Peak Memory: {full_context_results['avg_peak_memory']:.2f} MB")
log_result_no_prune(f"\n==={dataset_name} No Pruning (Full Context) ===")
log_result_no_prune(f"F1: {full_context_results['avg_f1']:.2f}, Latency: {full_context_results['avg_latency']:.2f}s, Peak Memory: {full_context_results['avg_peak_memory']:.2f} MB")

Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_gene


===narrativeqa  No Pruning (Full Context) ===
F1: 0.17, Latency: 11.71s,  Peak Memory: 16259.87 MB
